In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import numpy as np
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
import textwrap



start = time.time()
input_file = "результат по стране.xlsx"
output_file = "все цены.xlsx"
if not os.path.exists('graphs_2'):
    os.makedirs('graphs_2')

prices = pd.read_excel(input_file,
                    skiprows = 0,
                        dtype=str) 

prices_values = prices.drop(['Код товара', 'Наименование товара'], axis=1)

prices_values1 = prices_values.apply(lambda col:col.apply(lambda x: float(x.split(";")[0]) if pd.notna(x) else None))
# display(prices_values1)

prices_values2 = prices_values.apply(lambda col:col.apply(lambda x: str(x.split(";")[1]) if pd.notna(x) else None))
# display(prices_values2)


for index, row in prices_values1.iterrows():
    product_name = prices.loc[index, 'Наименование товара']
    product_name = product_name.replace('/', " ")
    product_name = product_name.replace('*', " ")
    product_name = product_name.replace('"', " ")
    row = row.dropna()
    
    rp_data = row[[col for col in row.index if 'РПЦ' in col]]
#     print(rp_data)
    pp_data = row[[col for col in row.index if 'АПЦ' in col]]
#     print(pp_data)


################################################# 


    try: 
        
### lof
        rp_n_neighbors = int(np.sqrt(len(rp_data) + 1))
        lof_rp = LocalOutlierFactor(n_neighbors = rp_n_neighbors)
        lof_outlier_scores_rp = -lof_rp.fit_predict(rp_data.values.reshape(-1, 1))
        lof_outlier_mask_rp = lof_outlier_scores_rp == 1
        lof_normal_mask_rp = lof_outlier_scores_rp == -1
        
        lof_max_normal_rp = rp_data[lof_normal_mask_rp].max()
        lof_min_normal_rp = rp_data[lof_normal_mask_rp].min()        

### if
        isf_rp = IsolationForest(contamination=0.2)
        isf_outlier_scores_rp = -isf_rp.fit_predict(rp_data.values.reshape(-1, 1))
        isf_outlier_mask_rp = isf_outlier_scores_rp == 1
        isf_normal_mask_rp = isf_outlier_scores_rp == -1
        
        isf_max_normal_rp = rp_data[isf_normal_mask_rp].max()
        isf_min_normal_rp = rp_data[isf_normal_mask_rp].min()

        ruo = max(isf_max_normal_rp, lof_max_normal_rp)
        rua = min(isf_max_normal_rp, lof_max_normal_rp)
        rla = max(isf_min_normal_rp, lof_min_normal_rp)
        rlo = min(isf_min_normal_rp, lof_min_normal_rp)
        
    except Exception:
        
        ruo = rp_data.max()
        rua = rp_data.max()
        rla = rp_data.min()
        rlo = rp_data.min()
        
        rp_n_neighbors = 0
        lof_outlier_mask_rp = []
        isf_outlier_mask_rp = []


    
    try: 
### lof
        pp_n_neighbors = int(np.sqrt(len(pp_data) + 1))
        lof_pp = LocalOutlierFactor(n_neighbors = pp_n_neighbors)
        lof_outlier_scores_pp = -lof_pp.fit_predict(pp_data.values.reshape(-1, 1))
        lof_outlier_mask_pp = lof_outlier_scores_pp == 1
        lof_normal_mask_pp = lof_outlier_scores_pp == -1
        
        lof_max_normal_pp = pp_data[lof_normal_mask_pp].max()
        lof_min_normal_pp = pp_data[lof_normal_mask_pp].min() 

### if
        isf_pp = IsolationForest(contamination=0.2)
        isf_outlier_scores_pp = -isf_pp.fit_predict(pp_data.values.reshape(-1, 1))
        isf_outlier_mask_pp = isf_outlier_scores_pp == 1
        isf_normal_mask_pp = isf_outlier_scores_pp == -1
        
        isf_max_normal_pp = pp_data[isf_normal_mask_pp].max()
        isf_min_normal_pp = pp_data[isf_normal_mask_pp].min()

        puo = max(isf_max_normal_pp, lof_max_normal_pp)
        pua = min(isf_max_normal_pp, lof_max_normal_pp)
        pla = max(isf_min_normal_pp, lof_min_normal_pp)
        plo = min(isf_min_normal_pp, lof_min_normal_pp)        
        
    except Exception:
        puo = pp_data.max()
        pua = pp_data.max()
        pla = pp_data.min()
        plo = pp_data.min()
        
        pp_n_neighbors = 0
        lof_outlier_mask_pp = []
        isf_outlier_mask_pp = []
        
    
    x_rp = list(range(1, len(rp_data) + 1))
    x_pp = list(range(len(rp_data) + 1, len(rp_data) + len(pp_data) + 1))
    
    
    # размер плота
    plt.figure(figsize=(18, 10))
    
    # перенос всех цен
    plt.scatter(x_rp, rp_data, c='blue', marker="o", s=6, label='РПЦ')
    plt.scatter(x_pp, pp_data, c='blue', marker="^", s=6, label='АПЦ')
    
    
    x_separator = len(x_rp) + 0.5
    
    plt.axvline(x=x_separator, color="black", linestyle="--", label="Разделитель", zorder=1)
    
    if ruo != rua:
        plt.fill_betweenx(y=[ruo, rua], x1=0, x2=x_separator, color="yellow", alpha=0.5, label="аномалии", zorder=0)
    plt.fill_betweenx(y=[rua, rla], x1=0, x2=x_separator, color="green", alpha=0.5, label="норм", zorder=0)
    if rla != rlo:
        plt.fill_betweenx(y=[rla, rlo], x1=0, x2=x_separator, color="yellow", alpha=0.5,  zorder=0)
    
    
    if puo != pua:
        plt.fill_betweenx(y=[puo, pua], x1=x_separator, x2=len(rp_data) + len(pp_data) + 1, 
                          color="yellow", alpha=0.5, zorder=0)
    plt.fill_betweenx(y=[pua, pla], x1=x_separator, x2=len(rp_data) + len(pp_data) + 1, 
                      color="green", alpha=0.5,  zorder=0)
    if pla != plo:
        plt.fill_betweenx(y=[pla, plo], x1=x_separator, x2=len(rp_data) + len(pp_data) + 1, 
                          color="yellow", alpha=0.5,  zorder=0)  


################################################# 

 
    plt.ylabel('Цена')
    title = f'График {product_name}'
    wrapped_title = '\n'.join(textwrap.wrap(title, width=60))
    plt.title(wrapped_title)
#     plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), fancybox=True, ncol=3, 
#                handles = [max_rp_legend, min_rp_legend, max_pp_legend, min_pp_legend])
    
    
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), fancybox=True, ncol=3)
    
    x_ticks = np.linspace(1, len(x_rp) + len(x_pp), dtype=int)
    plt.xticks(x_ticks)
    
#     x_ticks = list(range(1, len(x_rp) + len(x_pp) + 1 ))
#     plt.xticks(x_ticks)
    
    
#     plt.xticks(ticks=x_rp + x_pp, labels=x_rp + x_pp)
    plt.tight_layout()
       
    
    graph_filename = f'graphs_2/{product_name}.png'
    plt.savefig(graph_filename)
    
#     plt.show()
    plt.close()
    
        
    prices.loc[index, 'График'] = f'=HYPERLINK("{graph_filename}", "График {product_name}")'
    
    prices.loc[index, 'ruo'] = ruo
    prices.loc[index, 'rua'] = rua
    prices.loc[index, 'rla'] = rla
    prices.loc[index, 'rlo'] = rlo
    
    prices.loc[index, 'puo'] = puo
    prices.loc[index, 'pua'] = pua
    prices.loc[index, 'pla'] = pla
    prices.loc[index, 'plo'] = plo
    
prices.insert(2, 'puo', prices.pop('puo'))
prices.insert(2, 'pua', prices.pop('pua'))
prices.insert(2, 'pla', prices.pop('pla'))
prices.insert(2, 'plo', prices.pop('plo'))
        
prices.insert(2, 'ruo', prices.pop('ruo'))
prices.insert(2, 'rua', prices.pop('rua'))
prices.insert(2, 'rla', prices.pop('rla'))
prices.insert(2, 'rlo', prices.pop('rlo'))


prices.insert(2, 'График', prices.pop('График'))


# def add_rp_values(row):
#     for col in prices.columns:
#         if col.startswith('РПЦ'):
#             value = row[col]
#             if str(value) != "nan":
#                 value_list = str(value).split(";")
#                 value_list.extend([str(row["ruo"]), str(row["rua"]), str(row["rla"]), str(row["rlo"])])
#                 row[col] = ";".join(value_list)
#     return row

# prices = prices.apply(add_rp_values, axis=1)


# def add_pp_values(row):
#     for col in prices.columns:
#         if col.startswith('АПЦ'):
#             value = str(row[col])
#             if str(value) != "nan":
#                 value_list = value.split(";")
#                 value_list.extend([str(row["puo"]), str(row["pua"]), str(row["pla"]), str(row["plo"])])
#                 row[col] = ";".join(value_list)
#     return row

# prices = prices.apply(add_pp_values, axis=1)

prices_cols_to_melt = prices.columns[prices.columns.str.startswith("РПЦ") | prices.columns.str.startswith("АПЦ")]

prices_melted = pd.melt(prices, id_vars=["Код товара", "Наименование товара", "График", 
                                        "ruo", "rua", "rla", "rlo", "puo", "pua", "pla", "plo"],
                       value_vars = prices_cols_to_melt, var_name="Тип цены", value_name="Значение")

prices_melted = prices_melted.dropna(subset=['Значение'])

prices_melted[["Цена", "Конкурент", "Номер недели"]] = prices_melted['Значение'].str.split(";", expand=True)

prices_melted["Цена"] = prices_melted["Цена"].astype(float)

prices_melted = prices_melted.drop('Значение', axis=1)

prices_melted["Тип цены"] = prices_melted["Тип цены"].str.split().str[0]

def prices_determinition(row):
    if row["Тип цены"].startswith("РПЦ"):
#         print(row["rlo"])
        if row["Цена"] > row["ruo"]:
            return "1 ВВ"
        elif row["Цена"] > row["rua"]:
            return "2 ВА"
        elif row["Цена"] >= row["rla"]:
            return "3 Н"
        elif row["Цена"] >= row["rlo"]:
            return "4 НА"
        else:
            return "5 НВ"
    else:
        if row["Цена"] > row["puo"]:
            return "1 ВВ"
        elif row["Цена"] > row["pua"]:
            return "2 ВА"
        elif row["Цена"] >= row["pla"]:
            return "3 Н"
        elif row["Цена"] >= row["plo"]:
            return "4 НА"
        else:
            return "5 НВ"
        
        
prices_melted['Уровень'] = prices_melted.apply(prices_determinition, axis=1)



prices_melted['Средняя цена'] = (prices_melted.groupby(["Код товара", "Тип цены"])["Цена"].transform("mean").round(2))


condition = (prices_melted['Уровень'] != "5 НВ") & (prices_melted['Уровень'] != "1 ВВ")

prices_melted.loc[condition, "Средняя цена(без выбросов)"] = (prices_melted.loc[condition]
                                                              .groupby(["Код товара", "Тип цены"])["Цена"]
                                                              .transform("mean")
                                                              .round(2))

# prices_melted.loc["Средняя цена2"] = prices_melted.loc['Средняя цена2'].fillna("")
prices_melted["Откл от ср цены"] = (prices_melted["Цена"] - prices_melted['Средняя цена']) / (prices_melted['Средняя цена'])

prices_melted.to_excel(output_file, index=False)
print ("создан файл - " + str(time.time() - start))   


    

    
    

создан файл - 4.342455863952637
